In this notebook, we are interested in unsupervised learning. The goal in particular is to learn a linear projection of feature matrices retaining the largest fraction of information from the original matrix.

In [ ]:
# Load the required libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
from matplotlib.image import imread
#import skimage

# Compression with SVD

SVD constitutes the most basic method to reduce the dimensionality of a feature matrix $X\in\mathbb{R}^{d\times n}$ by seeking a rank-$k$ ($k < d$) linear projection with the smallest Frobenius (or spectral) norm. Mathematically, we are looking for $\hat{X}\in\mathbb{R}^{k\times n}$ satisfying
\begin{equation}
\min_{\mathrm{rk}(\hat{X})\leq k} \lVert X - \hat{X}\rVert_F^2,
\end{equation}
where $\mathrm{rk}(A)$ is the rank of $A$.
According to the **Eckart-Young theorem**, the global minimum of this optimization problem is reached for $\hat{X}$ given by the Singular Value Decomposition (SVD) of $X$, and retaining only the components associated to the $k$ largest singular values.

SVD states that any matrix $X$ can be decomposed as a set of three matrices
\begin{equation}
X = U S V^\mathrm{T},
\end{equation}
where $U$ and $V$ are orthonormal matrices ($U^\mathrm{T}U = V^\mathrm{T}V = I$), and
- $U \in \mathbb{R}^{d\times d}$ contains in columns the eigenvectors of $XX^\mathrm{T}$,
- $V \in \mathbb{R}^{n\times n}$ contains in columns the eigenvectors of $X^\mathrm{T}X$,
- $S \in \mathbb{R}^{d\times n}$ is a diagonal matrix containing the singular values $s_i$.

2 results about SVD are of importance for what follows. First, since both $S_\mathrm{L} = XX^\mathrm{T}$ and $S_\mathrm{R} = X^\mathrm{T}X$ are positive-definite matrices, all their eigenvalues are positive ($\forall i, \lambda_i^\mathrm{L} \geq 0, \lambda_i^\mathrm{R} \geq 0$). Second, if we order the eigenvalues of both matrices, then they are the same up to the order $m=\min(n,d)$. Above $m$, all the eigenvalues can be shown to be exactly zero. The singular values are defined as $s_i = \sqrt{\lambda_i^\mathrm{L}}$.

A more 'memory-efficient' (but equivalent) result takes only $m$ columns in $U$ and $V$, therefore removing the zero singular values.

#### How to find the singular values?

Let us focus on first on finding a rank-one approximation of a matrix $X$. Any rank-one matrix can be written as $\sigma u v^\mathrm{T}$. Then the goal is to solve
\begin{equation}
\operatorname*{argmin}_{\sigma, v, u} \lVert X - \sigma u v^\mathrm{T} \rVert^2_F,
\end{equation}
under the constraint that $\lVert u \rVert = \lVert v \rVert = 1$.

0. Perform this minimization. How would you then get a rank-two approximation and what would it give for $u_2$, $v_2$, and $\sigma_2$? Comment.

Let us first perform SVD of a single image of size $2000\times 1500$ taken from the ```DATA``` folder called ```dog.jpg```.


In [ ]:
plt.rcParams['figure.figsize'] = [16, 8]


A = imread(os.path.join('.','DATA','dog.jpg')) # Load the image
X = np.mean(A, -1); # Convert RGB to grayscale

img = plt.imshow(X)
img.set_cmap('gray')
plt.axis('off')
plt.show()

1. Checkout the [`np.linalg.svd`](https://docs.scipy.org/doc/numpy-1.10.4/reference/generated/numpy.linalg.svd.html) function, and **use it to decompose the input image $X$**. We will be using the 'full_matrices=False' option so that the matrices takes less memory space. **Check that the SVD gives back the full matrix.**

In [ ]:
# Your code to perform SVD

# Your code to check the shapes

# Let's transform S into a matrix
S = np.diag(S)
print('new S shape = ' + str(S.shape))

print('X=USV.T? ' + str(np.sum(np.mean(X-U@S@VT, axis=1)) < 1e-6 ))

2. Explore the compression with several values $r$ of singular values. **Explain what you get when varying $r$.**

In [ ]:
j = 0
for r in (5, 20, 100, 200):
    Xapprox = # Your code to construct the approximation
    plt.figure(j+1)
    j += 1
    img = plt.imshow(Xapprox)
    img.set_cmap('gray')
    plt.axis('off')
    plt.title('r = ' + str(r))
    plt.show()

3. Compute the cumulative sum of singular values, **how many dimensions are required to retain $80\%$ of the cumulative sum?**

In [ ]:
# Your code to plot and compute the cumulative sum of singular values

# [Eigenface](https://en.wikipedia.org/wiki/Eigenface)

The Eigenface dataset is a collection of facial images used computer vision and pattern recognition, particularly for facial recognition tasks. 
Here, we use a subset of it contained in the folder ```DATA/allFaces.mat```. It consists of facial images of $n = 38$ individuals under between 59 and 64 lighting conditions. Each image is $192\times168$, hence contains $32,256$ pixels.

In [ ]:
# Load the images
mat_contents = scipy.io.loadmat(os.path.join('.','DATA','allFaces.mat'))
print(mat_contents.keys())
faces = mat_contents['faces']
W = int(mat_contents['m'])
L = int(mat_contents['n'])
nfaces = np.ndarray.flatten(mat_contents['nfaces'])
print(L, W)

In [ ]:
# print the number of faces (lighting conditions) per individuals
print(nfaces)
print(faces.shape)

The list above gives the number of different lightings for each individual.
We will use the 36 first individuals as the training data, and keep the two last as test data.

Let us now plot the different images for each individual.

In [ ]:
allPersons = np.zeros((L*6,W*6))
count = 0

for j in range(6):
    for k in range(6):
        allPersons[j*L : (j+1)*L, k*W : (k+1)*W] = np.reshape(faces[:,np.sum(nfaces[:count])],(W,L)).T
        count += 1

img = plt.imshow(allPersons)
img.set_cmap('gray')
plt.axis('off')
plt.show()

Now we check for each person the different ligthing conditions.

In [ ]:
for person in range(len(nfaces)):
    subset = faces[:,sum(nfaces[:person]) : sum(nfaces[:(person+1)])]
    allFaces = np.zeros((L*8,W*8))
    
    count = 0
    
    for j in range(8):
        for k in range(8):
            if count < nfaces[person]:
                allFaces[j*L:(j+1)*L,k*W:(k+1)*W] = np.reshape(subset[:,count],(W,L)).T
                count += 1
                
    img = plt.imshow(allFaces)
    img.set_cmap('gray')
    plt.axis('off')
    plt.show()

4. Center the training dataset along the pixel dimension. **Visualise the average face**.

In [ ]:
# We use the first 36 people for training data
# We first compute the average face
trainingFaces = faces[:, :np.sum(nfaces[:36])] # size L*W=32256 by 2282
avgFace = # Your code to compute the average

# Center the training dataset along each pixel

# Plot the average face

5. Perform SVD on the centered dataset to find the matrices $U$, $S$, and $V$ that we will use to project the data. **What matrix should we use for projections? Visualise some eigenfaces (columns of the said matrix).**

In [ ]:
# Your code to perform SVD on the training data

In [ ]:
# Your code to check the shapes and visualise a mode of the SVD

Below we plot the first 100 eigenfaces.

In [ ]:
MATRIX_CHOICE = # Your choice of matrix
FirstEF = np.zeros((L*10,W*10))
count = 0
for j in range(10):
    for k in range(10):
        FirstEF[j*L : (j+1)*L, k*W : (k+1)*W] = np.reshape(MATRIX_CHOICE[:,count],(W,L)).T
        count += 1
        
img = plt.imshow(FirstEF)
img.set_cmap('gray')
plt.axis('off')
plt.show()

6. Compute the eigenface reconstruction on a test image for various order $r$. Comment.

In [ ]:
## Now show eigenface reconstruction of image that was omitted from test set

testFace = faces[:,np.sum(nfaces[:36])] # First face of person 37
plt.imshow(np.reshape(testFace,(W,L)).T)
plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

r_list = [25, 50, 100, 200, 400, 800, 1600]

for r in r_list:
    reconFace = # Your code to reconstruct the face
    img = plt.imshow(np.reshape(reconFace,(W,L)).T)
    img.set_cmap('gray')
    plt.title('r = ' + str(r))
    plt.axis('off')
    plt.show()

# Didn't you forget anything?

7. What do you think would happen for an image that is not a face at all? Test your intuition with the dog image from the beginning that we rescale below to the same size.

In [ ]:
import cv2
img = cv2.imread(os.path.join('.','DATA','dog.jpg'))
res = cv2.resize(img, dsize=(168, 192), interpolation=cv2.INTER_CUBIC)
X_s = np.mean(res, -1); # Convert RGB to grayscale

testFace = np.reshape(X_s.T,(-1,))
plt.imshow(np.reshape(testFace,(W,L)).T)
plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

In [ ]:
# Your code to reconstruct the new test image for various r

This kind of singular value decomposition is in fact very much used to recognize faces of different person. By projecting faces (with different light conditions or orientations, etc.) in the appropriate subspace, we suppress the 'noisy' variability in the data and retain only the essential information of the face. In the subspace, we can therefore compare two images and decide whether it is the same person.

8. Explore this idea by projecting two persons (say 2 and 7) in a 2D SVD space (say modes 4 and 5).

In [ ]:
## Project person 2 and 7 onto PC5 and PC6

P1num = 2 # Person number 2
P2num = 7 # Person number 7

P1 = faces[:,np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num])] # Selects all the faces from Person n°P1num
P2 = faces[:,np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num])] # Selects all the faces from Person n°P2num

PCAmodes = [4, 5] # Project onto PCA modes 4 and 5
# Your code to remove the average and project onto PCA modes


# Your code to visualise in the 2D projected space